In [1]:
import numpy as np
import pandas as pd

In [38]:
def relu(x):
    zero = np.zeros_like(x)
    return np.maximum([zero],[x])
    

In [39]:
def dx_relu(x):
    return(np.where(x>=0,1,0))

In [40]:
def identity(x):
    return x

In [41]:
def forward_propagation(x,w,activation):
    return activation(np.matmul(np.array(x),w))   

In [13]:
x_train = pd.DataFrame({
    'x_1':(0,0,1,1),
    'x_2':(0,1,0,1),
})

y_train = pd.DataFrame({
    'y_t':(0,1,1,0)
})
bias = pd.Series(np.ones_like(x_train.iloc[:,0]), name = 'bias')
x_train = pd.concat([x_train,bias],axis=1)

In [26]:
x_train = pd.concat([x_train,x_train],axis=0)

In [27]:
x_train.shape

(64, 3)

In [28]:
x_train = x_train.reset_index(drop=True)

In [29]:
x_train

,x_1,x_2,bias
0,0,0,1
1,0,1,1
2,1,0,1
3,1,1,1
4,0,0,1
5,0,1,1
6,1,0,1
7,1,1,1
8,0,0,1
9,0,1,1


In [35]:
y_train = pd.concat([y_train,y_train],axis=0)
y_train = y_train.reset_index(drop=True)

In [36]:
y_train

,y_t
0,0
1,1
2,1
3,0
4,0
5,1
6,1
7,0
8,0
9,1


In [55]:
i=0
j = np.array([2,2,1])
####weights
inputs = x_train.shape[1] ##inputs are the variables + bias
w_1 = np.random.normal(0,1,size = (inputs,j[0]))
w_2 = np.random.normal(0,1,size = (inputs,j[1]))
w_3 = np.random.normal(0,1,size = (inputs,j[2]))
lr = 0.001
activation_1= np.array(x_train)

while i < 200:

    ###layer 2

    activation_2 = forward_propagation(x_train,w_1,relu)
    n_2 = activation_2.shape[1]
    activation_2_ones = np.ones((n_2,1))
    activation_2 = np.hstack((activation_2[0],activation_2_ones))

    #####layer 3


    activation_3 = forward_propagation(activation_2,w_2,relu)
    n_3 = activation_3.shape[1]
    activation_3_ones = np.ones((n_3,1))
    activation_3 = np.hstack((activation_3[0],activation_3_ones))

    ####layer 4
    inputs_4 = activation_3.shape[1]
    activation_4 = forward_propagation(activation_3,w_3,identity)

    
    ####BackPropagation implementation 

    ### layer 4
    error_4 = np.array(activation_4 - y_train)
    d_w4 = np.matmul(activation_3[:,0:-1].T,error_4)
    d_b4 = np.matmul(activation_3[:,-1:].T,error_4)

    #### layer 3
    error_3 = np.matmul(error_4,w_3[0:-1].T)*dx_relu(activation_3[:,0:-1])
    d_w3 = np.matmul(activation_2[:,0:-1].T,error_3)
    d_b3 = np.matmul(activation_2[:,-1:].T,error_3)


    ### layer 2
    error_2 = np.matmul(error_3,w_2[0:-1].T)*dx_relu(activation_2[:,0:-1])
    d_w2 = np.matmul(activation_1[:,0:-1].T,error_2)
    d_b2 = np.matmul(activation_1[:,-1:].T,error_2)

    
    ####Updating weights

    #####update weights layer 2
    w_1[0:-1,] = w_1[0:-1,] -lr*d_w2
    w_1[-1:,] =  w_1[-1:,] - lr*d_b2
  #  print(w_1)

    #####update weights layer 3
    w_2[0:-1,] = w_2[0:-1,] -lr*d_w3
    w_2[-1:,] =  w_2[-1:,] - lr*d_b3

    ####update weights layer 4
    w_3[0:-1,] = w_3[0:-1,] -lr*d_w4
    w_3[-1:,] =  w_3[-1:,] - lr*d_b4
    if i % 100 == 0:
        print("MSE: ",error_4.mean())
    i+=1

print(activation_4)


MSE:  0.29848321142071876
MSE:  6.622108417175809e-10
[[0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]]
